# `R` function for using the Twitter v2 API

The function we use in this notebook is based on a [gist](https://gist.github.com/schochastics/1ff42c0211916d73fc98ba8ad0dcb261) by [David Schoch](https://gist.github.com/schochastics) which [Christopher Barrie](https://github.com/cjbarrie) used to create a small `R` package named [`academictwitteR`](https://github.com/cjbarrie/academictwitteR). As this package is only available via *GitHub* we need to install it first using the [`devtools` package](https://devtools.r-lib.org/). 

In [ ]:
devtools::install_github("cjbarrie/academictwitteR")

*Note*: The package `academictwitteR` includes two functions: one for collecting tweets for a list of users (which we will use in this notebook) and another one for collecting tweets for specific hashtags. While the original function by David Schoch is not as easy and straightforward to use as the functions from the `academictwitteR` package, it offers more versatility as it allows constructing more complicated [search queries](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query) using all options the Twitter API offers for this.

## Load packages

In addition to the `academictwitteR` package, we load the packages [`magrittr`](https://magrittr.tidyverse.org/index.html) and [`dplyr`](https://dplyr.tidyverse.org/) from the [`tidyverse`](https://www.tidyverse.org/) for some (minor) data wrangling.

In [ ]:
library(academictwitteR)
library(magrittr)
library(dplyr)

## Authentication

Before we can collect data via the Twitter v2 API, we need to set up our credentials. For this we need a [bearer token](https://developer.twitter.com/en/docs/authentication/oauth-2-0/bearer-tokens) for an app created for use with the v2 API (which you can get access to via the [academic research product track](https://developer.twitter.com/en/solutions/academic-research/products-for-researchers)).

**NB**: You should treat all information relating to your API access like a password and never share it or post it publicly anywhere. Although nobody except you should be able to access your personal instance of this notebook (and your edits will also not be persistent if you do not have/use a *GESIS Notebooks* user account), if you want to be extra cautious, you can delete your API access information from the following cell after running it once (and save the notebook again after that).

In [ ]:
# enter a bearer token for your v2 API app here
bearer_token <- ""

## Import list of accounts

The file [twitter_accounts.csv](./data/twitter_accounts.csv) in the `data` folder of this repository contains the Twitter screen names of [*GESIS - Leibniz Institute for the Social Sciences*](https://www.gesis.org/en/home) and the [*Social Data Science Lab*](http://socialdatalab.net/) at *Cardiff University* which we will use in the following examples.

In [ ]:
users_df <- read.csv("./data/twitter_accounts.csv")
users = as.character(users_df$Screen.Name)

## Search & collect tweets

We now collect all tweets sent from these accounts between January 1st and March 21st, 2021.

In [ ]:
tweets_df <- get_user_tweets(users,
                             "2021-01-01T00:00:00Z",
                             "2021-03-21T00:00:00Z",
                             bearer_token)

## User information

If we want to get some (additional) information about the accounts, we can use the `get_user_profiles` function.

In [ ]:
user_profiles <- get_user_profile(unique(tweets_df$author_id), bearer_token)

We can now combine the profile information with the tweets data. In the following cell, we just use the variables `name`, `username`, and `location` from the profile information. To see what else is included in that dataframe you could, e.g., run `names(user_profiles)` or `head(user_profiles)`.

In [ ]:
tweets_combined <- tweets_df %>% 
    left_join(select(user_profiles, id, name, username, location),
              by = c("author_id" = "id"))

## Save results

Finally, we can save our results as a `.csv` file.

In [ ]:
write.csv(tweets_combined, "./data/tweets_apiv2_r.csv")